In [2]:
import os
import xlrd
# import matplotlib.pyplot as plt
# import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import datetime
import statistics as stats
#from functools import reduce
#import math
from sklearn.preprocessing import LabelEncoder

### Daten einlesen und Spalten entfernen welche nicht benötigt werden.

In [ ]:
# messergebnisse_daten = pd.read_excel('./Messergebnisse.xlsx', skiprows=7, delimiter=';')
# messergebnisse_daten = messergebnisse_daten.rename(columns={'Messwert Innenmessschraube': 'Messwert Innenmessschraube',\
#                                                             'Messwert Messtaster': 'Messwert Messtaster', \
#                                                             'Klassifizierung nach Messchraube': 'Klassifizierung nach Messschraube', \
#                                                             'Klassifizierung nach Messtaster': 'Klassifizierung nach Messtaster', \
#                                                             'Klassifizierung nach Grenzlehrdorn': 'Klassifizierung nach Grenzlehrdorn'})

In [3]:
reib_daten = pd.read_csv('allLabeled.csv')
#messergebnisse_daten = messergebnisse_daten.drop(columns=['Unnamed: 6', 'Bezeichnung Bohrung'])
reib_daten = reib_daten.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Messschraube', 'Messtaster', 'Grenzlehrdorn'])

In [4]:
reib_daten

,12430012063.C1_Axis.Actual_Power,12430012063.Energy_Savings.Active_Power_Spindle1,12430012063.Energy_Savings.Active_Power_Z1,12430012063.Main_Spindle.Actual_Position_MCS,12430012063.Main_Spindle.Actual_Power,12430012063.Main_Spindle.Actual_Speed_Rate,12430012063.Main_Spindle.MPC_Peak,12430012063.Main_Spindle.MPC_Veff_Total,12430012063.Main_Spindle.Temperature_Spindle,12430012063.Maintenance_Package.Tool_Usage_Main_lubriTOOL,...,12430012063.X1_Axis.Actual_Feed_Rate,12430012063.X1_Axis.Actual_Position_MCS,12430012063.A1_Axis.Actual_Power,12430012063.Energy_Savings.Active_Power_Y1,12430012063.Y1_Axis.Actual_Feed_Rate,12430012063.Y1_Axis.Actual_Position_MCS,Ebene,Bohrung,12430012063.Maintenance_Package.Tool_Changer_Main,12430012063.Maintenance_Package.Tool_Changer_Main_lubriTOOL
0,1.500000,3.126359,53.527900,179.816218,0.500000,317.379310,1.000000,0.50,28.105000,74.39,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,NaN
1,1.571429,1.763218,-223.576625,184.350446,0.507463,317.500000,0.666667,0.50,NaN,NaN,...,1011.666667,367.525,NaN,NaN,NaN,NaN,1,2,NaN,NaN
2,1.000000,1.398177,382.341625,181.430268,0.500000,317.300000,2.333333,0.50,28.067143,74.38,...,3049.500000,384.030,NaN,NaN,NaN,NaN,1,3,NaN,NaN
3,1.500000,1.065278,700.204571,177.692973,0.492308,317.128205,1.666667,1.00,28.032857,NaN,...,0.000000,397.960,NaN,NaN,NaN,NaN,1,4,NaN,NaN
4,1.545455,0.225588,-132.687143,175.616696,0.508475,317.062500,1.500000,1.00,28.000000,74.37,...,4358.500000,410.085,NaN,NaN,NaN,NaN,1,5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,NaN,-16.873760,303.448700,180.035357,0.500000,951.461538,1.000000,0.75,NaN,NaN,...,1185.000000,410.090,NaN,NaN,NaN,NaN,25,77,NaN,NaN
806,NaN,-13.696250,627.496222,171.974828,0.500000,951.500000,1.600000,0.00,NaN,NaN,...,14.500000,425.330,NaN,NaN,NaN,NaN,25,78,NaN,NaN
807,NaN,-15.487385,56.574333,189.220345,0.533333,951.500000,1.750000,0.80,NaN,NaN,...,1742.500000,433.260,6.5,NaN,NaN,NaN,25,79,NaN,NaN
808,NaN,-16.962231,-276.777900,169.227931,0.500000,951.500000,0.500000,0.00,NaN,NaN,...,4252.000000,450.395,6.5,NaN,NaN,NaN,25,80,NaN,NaN


### Hilfestellung zur Ermittlung von Löchern
Die Funktion gibt die Intervalle für die gewünschten Bohrungen zurück. Zu beachten:
- keine negativen Zahlen
- Es muss für beide Funktionen dieselbe Ebene gewählt werden, da die Funktion ```bohrungenDavor()```mit 81 Bohrungen weniger rechnet!

In [ ]:
def bohrungEbene(ebene):
    if ebene == 1:
        return 81
    else:
        aktuelle_bohrung = 81 * ebene
    
    return aktuelle_bohrung

def bohrungenDavor(ebene):
    if ebene == 0:
        return 'cant have 0 layers'
    elif ebene == 1:
        return 0
    elif ebene > 1:
        bohrungen_davor = (ebene - 1) * 81
        return bohrungen_davor

    
print("Anz. Bohrungen davor: " + str(bohrungenDavor(2)) + "\nAnz. Bohrungen aktuell: " + str(bohrungEbene(2)))

In [ ]:
bohrungenDavor(25), bohrungEbene(25)

In [ ]:
#messergebnisse_per_layer = pd.DataFrame()

In [ ]:
#messergebnisse_per_layer = messergebnisse_per_layer.append(messergebnisse_daten[2025:2026])

In [ ]:
#messergebnisse_daten = messergebnisse_daten.append(messergebnisse_per_layer)

In [ ]:
#messergebnisse_per_layer

## Die Zelle benutzen wenn man die richtigen Messergebnisse mit Ebenen haben möchte!

In [5]:
#messergebnisse_per_layer.to_csv('./messergebnisse_erste.csv')
messergebnisse_daten = pd.read_csv('messergebnisse_erste.csv')
messergebnisse_daten.drop(columns=['Bezeichnung Bohrung', 'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 6'])
messergebnisse_daten = messergebnisse_daten.rename(columns={'Klassifizierung nach Messchraube': 'Klassifizierung nach Messschraube'})

### Bereinigen und hinzufügen Reibdaten

Hier werden die Spalten der Reibdaten mit values, welche mehr als 75% NaN sind, gelöscht. Danach werden die Reibdaten dem Gesamtdataframe hinzugefügt.

In [6]:
reib_daten = reib_daten.rename(columns=lambda x: x.strip('12430012063.'))

def calculatePercentileNaNInReibdaten(column):
    return (count_nans[column] / 81) * 100

count_nans = reib_daten.isna().sum()


for key, value in reib_daten.iteritems():
    if reib_daten[key].dtypes == 'float64':
        if calculatePercentileNaNInReibdaten(key) >= 75:
            reib_daten = reib_daten.drop(columns=key)

In [7]:
machine_data_layer = messergebnisse_daten.merge(reib_daten, left_index=True, right_index=True)

In [8]:
label_encoder = LabelEncoder()
label_encoder.fit(machine_data_layer['Klassifizierung nach Messtaster'])
machine_data_layer['Klassifizierung nach Messtaster'] = label_encoder.\
        transform(machine_data_layer['Klassifizierung nach Messtaster'])

label_encoder.fit(machine_data_layer['Klassifizierung nach Grenzlehrdorn'])
machine_data_layer['Klassifizierung nach Grenzlehrdorn'] = label_encoder.\
        transform(machine_data_layer['Klassifizierung nach Grenzlehrdorn'])

label_encoder.fit(machine_data_layer['Klassifizierung nach Messschraube'])
machine_data_layer['Klassifizierung nach Messschraube'] = label_encoder.\
        transform(machine_data_layer['Klassifizierung nach Messschraube'])

In [9]:
machine_data_layer

,Unnamed: 0,Bezeichnung Bohrung,Klassifizierung nach Grenzlehrdorn,Klassifizierung nach Messschraube,Klassifizierung nach Messtaster,Messwert Innenmessschraube,Messwert Messtaster,Unnamed: 0.1,Unnamed: 6,Energy_Savings.Active_Power_Spindle,...,Z1_Axis.Actual_Feed_Rate,Z1_Axis.Actual_Position_MCS,Z1_Axis.Actual_Power,timeStamp,Energy_Savings.Active_Power_X,Tool_Control_Center.Axial_Force_Compression,X1_Axis.Actual_Feed_Rate,X1_Axis.Actual_Position_MCS,Ebene,Bohrung
0,0,NaN,0,0,1,12.016,12.019,0.0,NaN,3.126359,...,354.875000,-304.447544,13.739130,2019-11-20 09:34:30.170,NaN,NaN,NaN,NaN,1,1
1,1,NaN,0,0,0,12.016,12.018,1.0,NaN,1.763218,...,8.800000,-302.581802,13.766667,2019-11-20 09:34:53.847,416.853000,0.052632,1011.666667,367.525,1,2
2,2,NaN,0,0,0,12.016,12.015,2.0,NaN,1.398177,...,-156.000000,-302.535946,12.562500,2019-11-20 09:35:16.250,-60.904000,0.048148,3049.500000,384.030,1,3
3,3,NaN,0,0,0,12.016,12.015,3.0,NaN,1.065278,...,26.000000,-303.418349,17.350000,2019-11-20 09:35:38.677,-111.657000,0.054286,0.000000,397.960,1,4
4,4,NaN,0,0,0,12.016,12.015,4.0,NaN,0.225588,...,67.909091,-302.540270,16.608696,2019-11-20 09:36:00.879,4.060500,0.052000,4358.500000,410.085,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,805,NaN,0,0,1,12.014,12.024,2021.0,NaN,-16.873760,...,-72.166667,-323.262222,13.600000,2019-12-16 10:37:50.743,0.000000,0.150000,1185.000000,410.090,25,77
806,806,NaN,0,0,1,12.013,12.022,2022.0,NaN,-13.696250,...,-9.666667,-323.293333,17.700000,2019-12-16 10:37:56.565,-261.210000,0.150000,14.500000,425.330,25,78
807,807,NaN,0,0,1,12.012,12.026,2023.0,NaN,-15.487385,...,249.333333,-319.254444,24.181818,2019-12-16 10:38:02.202,16.692333,0.150000,1742.500000,433.260,25,79
808,808,NaN,0,0,1,12.014,12.030,2024.0,NaN,-16.962231,...,304.222222,-320.037857,26.875000,2019-12-16 10:38:07.944,418.206500,0.150000,4252.000000,450.395,25,80


In [10]:
machine_data_layer = machine_data_layer.drop(columns=['Bezeichnung Bohrung', 'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 6', 'Bohrung'])

In [11]:
machine_data_layer

,Klassifizierung nach Grenzlehrdorn,Klassifizierung nach Messschraube,Klassifizierung nach Messtaster,Messwert Innenmessschraube,Messwert Messtaster,Energy_Savings.Active_Power_Spindle,Energy_Savings.Active_Power_Z,Main_Spindle.Actual_Position_MCS,Main_Spindle.Actual_Power,Main_Spindle.Actual_Speed_Rate,...,Y1_Axis.Actual_Power,Z1_Axis.Actual_Feed_Rate,Z1_Axis.Actual_Position_MCS,Z1_Axis.Actual_Power,timeStamp,Energy_Savings.Active_Power_X,Tool_Control_Center.Axial_Force_Compression,X1_Axis.Actual_Feed_Rate,X1_Axis.Actual_Position_MCS,Ebene
0,0,0,1,12.016,12.019,3.126359,53.527900,179.816218,0.500000,317.379310,...,0.500000,354.875000,-304.447544,13.739130,2019-11-20 09:34:30.170,NaN,NaN,NaN,NaN,1
1,0,0,0,12.016,12.018,1.763218,-223.576625,184.350446,0.507463,317.500000,...,0.500000,8.800000,-302.581802,13.766667,2019-11-20 09:34:53.847,416.853000,0.052632,1011.666667,367.525,1
2,0,0,0,12.016,12.015,1.398177,382.341625,181.430268,0.500000,317.300000,...,0.600000,-156.000000,-302.535946,12.562500,2019-11-20 09:35:16.250,-60.904000,0.048148,3049.500000,384.030,1
3,0,0,0,12.016,12.015,1.065278,700.204571,177.692973,0.492308,317.128205,...,1.000000,26.000000,-303.418349,17.350000,2019-11-20 09:35:38.677,-111.657000,0.054286,0.000000,397.960,1
4,0,0,0,12.016,12.015,0.225588,-132.687143,175.616696,0.508475,317.062500,...,0.666667,67.909091,-302.540270,16.608696,2019-11-20 09:36:00.879,4.060500,0.052000,4358.500000,410.085,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,0,0,1,12.014,12.024,-16.873760,303.448700,180.035357,0.500000,951.461538,...,1.000000,-72.166667,-323.262222,13.600000,2019-12-16 10:37:50.743,0.000000,0.150000,1185.000000,410.090,25
806,0,0,1,12.013,12.022,-13.696250,627.496222,171.974828,0.500000,951.500000,...,0.857143,-9.666667,-323.293333,17.700000,2019-12-16 10:37:56.565,-261.210000,0.150000,14.500000,425.330,25
807,0,0,1,12.012,12.026,-15.487385,56.574333,189.220345,0.533333,951.500000,...,0.750000,249.333333,-319.254444,24.181818,2019-12-16 10:38:02.202,16.692333,0.150000,1742.500000,433.260,25
808,0,0,1,12.014,12.030,-16.962231,-276.777900,169.227931,0.500000,951.500000,...,0.600000,304.222222,-320.037857,26.875000,2019-12-16 10:38:07.944,418.206500,0.150000,4252.000000,450.395,25


In [12]:
# machine_data_layer['Main_Spindle.Temperature_Spindle'] = machine_data_layer['Main_Spindle.Temperature_Spindle'].fillna(method='ffill')
# machine_data_layer['Y1_Axis.Temperature_Y_Slide'] = machine_data_layer['Y1_Axis.Temperature_Y_Slide'].fillna(method='ffill')
# machine_data_layer['Tool_Control_Center.Axial_Force_Compression'] = machine_data_layer['Tool_Control_Center.Axial_Force_Compression'].fillna(method='pad')
# machine_data_layer['A1_Axis.Actual_Power'] = machine_data_layer['A1_Axis.Actual_Power'].fillna(method='pad')
# machine_data_layer['Tool_Control_Center.Axial_Force_Tension'] = machine_data_layer['Tool_Control_Center.Axial_Force_Tension'].fillna(method='pad')
# machine_data_layer['Tool_Control_Center.Bending_Moment'] = machine_data_layer['Tool_Control_Center.Bending_Moment'].fillna(method='pad')
for x in machine_data_layer:
    if machine_data_layer[x].isna().any():
        machine_data_layer[x] = machine_data_layer[x].fillna(method='bfill')


machine_data_layer.isna().any()

Klassifizierung nach Grenzlehrdorn             False
Klassifizierung nach Messschraube              False
Klassifizierung nach Messtaster                False
Messwert Innenmessschraube                     False
Messwert Messtaster                            False
Energy_Savings.Active_Power_Spindle            False
Energy_Savings.Active_Power_Z                  False
Main_Spindle.Actual_Position_MCS               False
Main_Spindle.Actual_Power                      False
Main_Spindle.Actual_Speed_Rate                 False
Main_Spindle.MPC_Peak                          False
Main_Spindle.MPC_Veff_Total                    False
Tool_Control_Center.Bending_Moment             False
X1_Axis.Actual_Power                           False
Y1_Axis.Actual_Power                           False
Z1_Axis.Actual_Feed_Rate                       False
Z1_Axis.Actual_Position_MCS                    False
Z1_Axis.Actual_Power                           False
timeStamp                                     

In [30]:
# write to csv
machine_data_layer.to_csv('./Messergebnisse_merged.csv')
machine_data_layer.to_json('./Messergebnisse_merged.json', orient='split')